# Whole notebook in a nutshell

bash /scratch/iss_decoding/nf/workflow-segmentation/local_nuc_seg.sh \
    /scratch/tl10/segmentation/seg.yaml

In [1]:
! cat /scratch/tl10/segmentation/seg.yaml

csv : /scratch/iss_decoding/data/segmentation/in/seg.csv
out_dir : ./out/
object_diameter : [70]
expand_in_pixel : [10, 15]

sif_container : /scratch/iss_decoding/sifs/large_cellseg.sif


In [1]:
! cat /scratch/iss_decoding/data/segmentation/in/seg.csv

index,filepath
0,/scratch/iss_decoding/data/registration/out/demo_optflow_seg_optflow_reg_result_stack.tif


# Or step by step ...

CellPose tutorial
https://github.com/MouseLand/cellpose/blob/main/notebooks/run_cellpose_GPU.ipynb

In [1]:
! nvidia-smi

Mon Jun 26 13:21:43 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 495.46       CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:00:05.0 Off |                  N/A |
| 28%   10C    P8     8W / 250W |      2MiB / 11178MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:00:0C.0 Off |                  N/A |
| 28%   

In [2]:
import napari
from dask_image.imread import imread

In [3]:
# Set up root directory
root = "/scratch/iss_decoding/workshop_data/segmentation/in/"
out_dir = "./out"

In [4]:
! mkdir "{out_dir}"

mkdir: cannot create directory ‘./out’: File exists


In [5]:
ref_DAPI = imread(f"{root}/demo_optflow_seg_optflow_reg_result_stack.tif")[0].squeeze()
ref_DAPI.shape

(4114, 4369)

In [6]:
viewer = napari.Viewer()

In [7]:
viewer.add_image(ref_DAPI)

<Image layer 'ref_DAPI' at 0x7f9fea813b80>

In [8]:
from cellpose import models, core
import tifffile as tf
import matplotlib.pyplot as plt

use_GPU = core.use_gpu()
print('>>> GPU activated? %d'%use_GPU)

>>> GPU activated? 1


In [9]:
# model = models.Cellpose(gpu=use_GPU, model_type='cyto2') # Initialize model
model = models.Cellpose(gpu=False, model_type='cyto2') # Initialize model, if GPU has issues

In [10]:
channels = [[0, 0]] # Use just DAPI channel for segmentation

In [11]:
ref_DAPI.shape

(4114, 4369)

In [12]:
masks, flows, styles, diams = model.eval(ref_DAPI,
                                         diameter=60,
                                         flow_threshold=None,
                                         channels=channels) # Run segmentation

In [14]:
viewer.add_image(masks)
viewer.add_labels(masks)

<Labels layer 'masks [3]' at 0x7f9db1345430>

In [15]:
from skimage.segmentation import expand_labels

In [16]:
expanded_masks = expand_labels(masks, distance=10)

In [17]:
viewer.add_labels(expanded_masks)

<Labels layer 'expanded_masks' at 0x7f9db10bfdc0>

In [18]:
tf.imwrite(f"{out_dir}/cellpose_segmentation.tif", expanded_masks)